In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,42282
2,Huelva,Confirmados PDIA 14 días,2717
3,Huelva,Tasa PDIA 14 días,"529,4541769783892"
4,Huelva,Confirmados PDIA 7 días,1128
5,Huelva,Tasa PDIA 7 dias,"219,81019934914355"
6,Huelva,Total Confirmados,42482
7,Huelva,Curados,35580
8,Huelva,Fallecidos,396


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  42282.0


/tmp/ipykernel_17417/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11797.0


/tmp/ipykernel_17417/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_17417/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_17417/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_17417/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 431 personas en los últimos 7 días 

Un positivo PCR cada 198 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,42282.0,1128.0,2717.0,513170.0,219.810199,529.454177,512.0
Huelva-Costa,24868.0,734.0,1607.0,289548.0,253.498556,555.002970,322.0
Condado-Campiña,13279.0,319.0,883.0,156231.0,204.184829,565.188727,148.0
Huelva (capital),11797.0,334.0,725.0,143837.0,232.207290,504.042771,131.0
Lepe,2797.0,99.0,216.0,27880.0,355.093257,774.748924,47.0
Isla Cristina,2864.0,82.0,172.0,21393.0,383.302950,804.001309,44.0
Cartaya,1698.0,62.0,109.0,20083.0,308.718817,542.747597,37.0
Ayamonte,1832.0,75.0,160.0,21104.0,355.382866,758.150114,35.0
Sierra de Huelva-Andévalo Central,3741.0,66.0,212.0,67391.0,97.935926,314.582066,33.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Sanlúcar de Guadiana,31.0,4.0,21.0,403.0,992.555831,5210.918114,1.0
Cañaveral de León,18.0,3.0,7.0,396.0,757.575758,1767.676768,3.0
Corteconcepción,43.0,4.0,9.0,536.0,746.268657,1679.104478,0.0
Nava (La),7.0,1.0,4.0,258.0,387.596899,1550.387597,1.0
Escacena del Campo,206.0,8.0,34.0,2287.0,349.803236,1486.663752,3.0
Villarrasa,214.0,8.0,26.0,2211.0,361.827227,1175.938489,1.0
Campofrío,18.0,3.0,8.0,713.0,420.757363,1122.019635,2.0
Bonares,435.0,21.0,59.0,6060.0,346.534653,973.597360,13.0
Villablanca,256.0,11.0,28.0,2885.0,381.282496,970.537262,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cumbres Mayores,121.0,1.0,13.0,1749.0,57.175529,743.281875,1.0,0.076923
Calañas,240.0,1.0,11.0,2768.0,36.127168,397.398844,1.0,0.090909
Nerva,242.0,1.0,10.0,5169.0,19.346102,193.461018,1.0,0.100000
Alosno,221.0,1.0,7.0,3933.0,25.425884,177.981185,0.0,0.142857
Minas de Riotinto,166.0,3.0,16.0,3812.0,78.698846,419.727177,1.0,0.187500
Sanlúcar de Guadiana,31.0,4.0,21.0,403.0,992.555831,5210.918114,1.0,0.190476
Villanueva de los Castillejos,190.0,1.0,5.0,2825.0,35.398230,176.991150,0.0,0.200000
Zarza-Perrunal (La),47.0,1.0,5.0,1253.0,79.808460,399.042298,0.0,0.200000
Paymogo,85.0,1.0,5.0,1162.0,86.058520,430.292599,0.0,0.200000
